In [ ]:
#sentences = "The incident management process start with the detection of a service failure".split(".")
sentences = "My name is sing song as well. My am a mother as well as tall. I am happy. You sing like my mother".split(".")
search_keywords=['as','as']

for sentence in sentences:
    print("{} key words in sentence:".format(sum(1 for word in search_keywords if word in sentence)))
    print(sentence + "\n")

#===================================
dct = {}
for sentence in sentences:
    dct[sentence] = sum(1 for word in search_keywords if word in sentence)
best_sentences = [key for key,value in dct.items() if value == max(dct.values())]
print("\n".join(best_sentences))

# Outputs:
#My name is sing song
# You sing like my mother
#====================================
if (any(map(lambda word: word in sentence, search_keywords))):
    print ('sentence :', sentence)

#====================================
sent_words = []
for sentence in sentences:
    sent_words.append(set(sentence.split()))
num_keywords = [len(sent & set(search_keywords)) for sent in sent_words]

# Find only one sentence
ind = num_keywords.index(max(num_keywords))
# Find all sentences with that number of keywords
ind = [i for i, x in enumerate(num_keywords) if x == max(num_keywords)]

2 key words in sentence:
My name is sing song as well

2 key words in sentence:
 My am a mother as well as tall

0 key words in sentence:
 I am happy

0 key words in sentence:
 You sing like my mother

My name is sing song as well
 My am a mother as well as tall


In [ ]:
import re

templates = [
    "I have four {fruit} in {place}",
    "I have four {fruit} and {grain} in {place}",
]

my_dict = {'fruit': ['apple', 'banana', 'mango'],
           'place': ['kitchen', 'living room'],
           'grain' : ['wheat', 'rice']
           }

words = {k: '(?P<{}>{})'.format(k, '|'.join(map(re.escape, v))) for k, v in my_dict.items()}
patterns = [re.compile(template.format(**words)) for template in templates]

def find_matches(sentence):
    for pattern in patterns:
        match = pattern.match(sentence)
        if match:
            return match.groupdict()

print(find_matches("I have four apple in kitchen"))

{'fruit': 'apple', 'place': 'kitchen'}


In [ ]:
import os, sys
import spacy
from spacy.matcher import Matcher

text = "Receipt department returns the good to the vendor and system sends notification to the purchase department"

SUBJECTS = ["nsubj", "nsubjpass", "csubj", "csubjpass", "agent", "expl"]
VERB = ["ROOT"]
OBJECTS = ["dobj", "dative", "attr", "oprd"]
bpmn_label = []

nlp = spacy.load("en_core_web_sm")
doc = nlp(text)

def isNegated(tok):
    negations = {"no", "not", "n't", "never", "none"}
    for dep in list(tok.lefts) + list(tok.rights):
        if dep.lower_ in negations:
            return True
    return False

def getSubsFromConjunctions(subs):
    moreSubs = []
    for sub in subs:
        # rights is a generator
        rights = list(sub.rights)
        rightDeps = {tok.lower_ for tok in rights}
        if "and" in rightDeps:
            moreSubs.extend([tok for tok in rights if tok.dep_ in SUBJECTS or tok.pos_ == "NOUN"])
            if len(moreSubs) > 0:
                moreSubs.extend(getSubsFromConjunctions(moreSubs))
                #print('get more subjects', moreSubs)
    return moreSubs

def findSubs(tok):
    head = tok.head
    pos = tok.pos_
    while head.pos_ != "VERB" and head.pos_ != "NOUN" and head.head != head:
        head = head.head
    if head.pos_ == "VERB":
        subs = [tok for tok in head.lefts if tok.dep_ in SUBJECTS]
        if len(subs) > 0:
            verbNegated = isNegated(head)
            subs.extend(getSubsFromConjunctions(subs))
            return subs, verbNegated
        elif head.head != head:
            return findSubs(head)
    elif head.pos_ == "NOUN":
        return [head], isNegated(tok)
    return [], False

def findSubject_with_negated(tokens):
    subs = []
    verbs = [tok for tok in tokens if tok.pos_ == "VERB" and tok.dep_ != "aux"]
    for v in verbs:
        verbNegated = isNegated(v)
        subject = [tok for tok in tokens.lefts if tok.dep_ in SUBJECTS]
        
        #ถ้าหาไม่เจอ ให้ทำต่อ
        if len(subject) > 0:
            subs.extend(getSubsFromConjunctions(subject))
        else:
            foundSubs, verbNegated = findSubs(v)
            subs.extend(foundSubs)
    print('---SUBJECT---', subs)

def findVerb(tokens):
    subject = [tok for tok in tokens if tok.dep_ in VERB]
    print("---VERB---", subject)

def findObject(tokens):
    verb = [tok for tok in tokens if tok.dep_ in OBJECTS]
    print("---OBJECT---", verb)

all_match = []
def descriptive_match_pattern(text):
    nlp = spacy.load('en_core_web_sm')
    matcher = Matcher(nlp.vocab)
    doc = nlp(text)
        
    #VERB + PREP + POBJ
    verb_dobj_patterns = [
        {'POS': 'VERB'},
        {'POS': 'DET', 'OP':'*'},
        {'DEP': 'dobj'}
    ]

    verb_dobj_to_patterns = [
        {'POS': 'VERB'},
        {'POS': 'DET', 'OP':'*'},
        {'DEP': 'dobj'},
        {'DEP': 'prep'},
        {'POS': 'NOUN'}
    ]

    matcher.add('TASK-EVENT', None, verb_dobj_patterns)
    matcher.add('TASK-EVENT', None, verb_dobj_to_patterns)
    matches = matcher(doc)
    
    for match in matches:
        match_id, start, end = match
        string_id = nlp.vocab.strings[match_id]
        span = doc[start:end]
        print(string_id, span.text)
        all_match.append(span.text)

    my_match_list = list(dict.fromkeys(all_match))
    print(my_match_list)

all_subs = [] 
def find_left_and_right(doc):   
    index = 0
    nounIndices = []

    noun_chunks_list = []
    for np in doc.noun_chunks: 
        noun_chunks_list.append(str(np))

    for token in doc:
        if token.pos_ == 'NOUN':
            nounIndices.append(index)
        index = index + 1

    for idxValue in nounIndices:
        doc = nlp(text)
        span = doc[doc[idxValue].left_edge.i : doc[idxValue].right_edge.i+1]
        span.merge()

        for token in doc:
            if token.pos_ == "VERB":
                subs = [tok for tok in token.rights if tok.dep_ in "dobj"]
                all_subs.append(token.text + " " +str(subs).replace('[','').replace(']',''))
    
    #TASK EVENT CLUSTER 
    remove_repeately_list_with_dictionary()


def remove_repeately_list_with_dictionary():
    my_list = list(dict.fromkeys(all_subs))
    descriptive_match_pattern(str(my_list))

#START
find_left_and_right(doc)

TASK-EVENT sends notification
TASK-EVENT returns the good
['sends notification', 'returns the good']


In [5]:
!pip install textacy

     |████████████████████████████████| 184kB 6.1MB/s 
     |████████████████████████████████| 1.9MB 39.8MB/s 
     |████████████████████████████████| 102kB 11.1MB/s 
     |████████████████████████████████| 481kB 44.8MB/s 
  Created wheel for cytoolz: filename=cytoolz-0.11.0-cp36-cp36m-linux_x86_64.whl size=1225578 sha256=517b4f97d640380ebb1ac9114a401eb13151302470af2a979cb252a7aa08f9e7
  Stored in directory: /root/.cache/pip/wheels/a1/32/3c/9c9926b510647cacdde744b2c7acdf1ccd5896fbb7f8d5df0c
Successfully built cytoolz


In [6]:
import spacy
from spacy.symbols import nsubj, VERB
import textacy

def findSubjectWithAncestor(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    root = [token for token in doc if token.head == token][0]
    subject = list(root.lefts)[0]
    for descendant in subject.subtree:
        assert subject is descendant or subject.is_ancestor(descendant)
        print(descendant.text, descendant.dep_, descendant.n_lefts,
              descendant.n_rights,
              [ancestor.text for ancestor in descendant.ancestors])

def findSubjectVerbsWithChildren(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)

    verbs = []
    for possible_verb in doc:
        print(possible_verb.tag_, possible_verb.text)
        if possible_verb.pos == VERB:
            for possible_subject in possible_verb.children:
                if possible_subject.dep == nsubj:
                    verbs.append(possible_verb)
                    break
    print(verbs)

def findNounPhrasesVerbs(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)

    #VERB PHRASE
    patterns = [{"POS":"ADV"},{"POS":"VERB"}]

    # Analyze syntax
    print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
    print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

    # Find named entities, phrases and concepts
    for entity in doc.ents:
        print(entity.text, entity.label_)

if __name__ == "__main__":
    findNounPhrasesVerbs("the sales department receives an order")
    findSubjectVerbsWithChildren("the sales department receives an order")

Noun phrases: ['the sales department', 'an order']
Verbs: ['receive']
DT the
NNS sales
NN department
VBZ receives
DT an
NN order
[receives]


"\n(base) Toppees-MacBook-Pro:~ toppee$ python /Users/toppee/Desktop/test2.py\nNoun phrases: ['we <NP>', 'a new order <NP>', 'the sales department <NP>', 'my masters <NP>', 'the necessary parts <NP>', 'quantities <NNS>', 'the delivery date <NP>']\nVerbs: ['get <VBP>', 'determine <VBZ>']\n(base) Toppees-MacBook-Pro:~ toppee$ \n"